In [1]:
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer, Metadata, Interpreter

In [2]:
def train_nlu(data, configuration, model_dir):
    training_data = load_data(data)
    trainer = Trainer(config.load(configuration))
    trainer.train(training_data)
    model_directory = trainer.persist(model_dir,fixed_model_name='faq_chatbot_nlu')

In [20]:
train_nlu_model_switch = True

In [21]:
if (train_nlu_model_switch):
    train_nlu('./data/data.json','spacy_configuration.json','./models/faq_chatbot_nlu')

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished


In [22]:
def run_nlu(query):
    interpreter = Interpreter.load('./models/faq_chatbot_nlu/default/faq_chatbot_nlu')
    return interpreter.parse(query)

In [23]:
query = "Thanks!"

In [16]:
parsed_query = run_nlu(query)
parsed_query

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/faq_chatbot_nlu/default/faq_chatbot_nlu\entity_synonyms.json'
  "".format(entity_synonyms_file))


{'entities': [],
 'intent': {'confidence': 0.53561186733242661, 'name': 'gratitude'},
 'intent_ranking': [{'confidence': 0.53561186733242661, 'name': 'gratitude'},
  {'confidence': 0.40079041617131533, 'name': 'greet'},
  {'confidence': 0.063597716496257978, 'name': 'goodbye'}],
 'text': 'Thanks!'}

In [19]:
run_nlu("Good morning")

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/faq_chatbot_nlu/default/faq_chatbot_nlu\entity_synonyms.json'
  "".format(entity_synonyms_file))


{'entities': [],
 'intent': {'confidence': 0.90318292861888216, 'name': 'greet'},
 'intent_ranking': [{'confidence': 0.90318292861888216, 'name': 'greet'},
  {'confidence': 0.053229410377504179, 'name': 'goodbye'},
  {'confidence': 0.043587661003613637, 'name': 'gratitude'}],
 'text': 'Good morning'}

In [24]:
run_nlu("where do you put up")

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/faq_chatbot_nlu/default/faq_chatbot_nlu\entity_synonyms.json'
  "".format(entity_synonyms_file))


{'entities': [],
 'intent': {'confidence': 0.33561563891876722, 'name': 'location'},
 'intent_ranking': [{'confidence': 0.33561563891876722, 'name': 'location'},
  {'confidence': 0.29495086343843768, 'name': 'goodbye'},
  {'confidence': 0.27402788958955021, 'name': 'greet'},
  {'confidence': 0.095405608053245147, 'name': 'gratitude'}],
 'text': 'where do you put up'}